# Configure database connection

We have prepared a hosted database (triplestore / SPARQL endpoint) for you. Let’s connect to it:

In this Notebook, go to "Secrets" in the left sidebar, and add two secrets:

* Name: `sparql_user`, Value: your username (from the note)
* Name: `sparql_password`, Value: your password (from the note)

Enable "Notebook access" for both.

Then execute the following cell to set the database connection details as environment variables:

In [ ]:
import os
from google.colab import userdata
os.environ['SPARQL_USER'] = userdata.get('sparql_user')
os.environ['SPARQL_PASSWORD'] = userdata.get('sparql_password')
os.environ['SPARQL_ENDPOINT_STORE'] = "https://sd-84c02130.stardog.cloud:5820/" + userdata.get('sparql_user')
os.environ['SPARQL_ENDPOINT_QUERY'] = "https://sd-84c02130.stardog.cloud:5820/" + userdata.get('sparql_user') + "/query"
os.environ['SPARQL_ENDPOINT_UPDATE'] = "https://sd-84c02130.stardog.cloud:5820/" + userdata.get('sparql_user') + "/update"


## Test connection

To test if the database connection works, execute the following SPARQL query. It should output just the string `graph`, which means that no named graphs exist yet.

In [ ]:
!curl -X GET \
      -H "Accept:text/csv" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      "${SPARQL_ENDPOINT_QUERY}" \
      --data-urlencode "query=SELECT DISTINCT ?graph WHERE { GRAPH ?graph { [] ?p [] . } }"

# Download files

In [ ]:
!git clone https://github.com/brox-it/dwt24-kg-notebook.git
%cd ./dwt24-kg-notebook/

# Install tools

## SPARQL Anything

Links: [Website](https://sparql-anything.cc/) · [Repo](https://github.com/SPARQL-Anything/sparql.anything) · [Documentation](https://sparql-anything.readthedocs.io/en/latest/)

In [ ]:
!wget https://github.com/SPARQL-Anything/sparql.anything/releases/download/0.9.0/sparql-anything-0.9.0.jar -O sparql-anything.jar

## PySHACL

Links: [Repo](https://github.com/RDFLib/pySHACL) · [PyPI](https://pypi.org/project/pyshacl/)

In [ ]:
%pip install pyshacl

# Build the knowledge graph

## 1) Understand the data (XML)

The input data is about

* persons,
* the management jobs they had, and
* the skills/competences they used in these jobs.

Each job can be assigned to a job category. Some skills have related skills.

The data is in four XML files in the `transformations_input` directory:

* `distances.xml` for skills that are related to each other
* `jobcategories.xml` for assigning job categories to jobs
* `jobcategories-skills.xml` for connecting job categories and skills
* `skills.xml` for jobs (with skills) of a person


## 2) Create the ontology (RDF)


The ontology describes what the terms (e.g., classes and properties) *mean*.

The RDF file (in Turtle) is in the `ontologies` directory.

## 3) Transform the data (XML to RDF)

There are many ways to convert data to RDF. One of these ways is SPARQL-based. SPARQL is the query language for RDF, but there are tools that allow querying even *non-RDF* data as if it already were RDF. One of these tools is **SPARQL Anything**.

The four query files (one for each XML file) are in the `transformations` directory.

The generated RDF files (in Turtle) get saved in the `transformations_output` directory.

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/distances.rq" \
      --configuration "location=./transformations_input/distances.xml" \
      --output "./transformations_output/distances.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/jobcategories.rq" \
      --configuration "location=./transformations_input/jobcategories.xml" \
      --output "./transformations_output/jobcategories.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/jobcategories-skills.rq" \
      --configuration "location=./transformations_input/jobcategories-skills.xml" \
      --output "./transformations_output/jobcategories-skills.ttl" \
      --format "TTL"

In [ ]:
!java -jar sparql-anything.jar \
      --query "./transformations/skills.rq" \
      --configuration "location=./transformations_input/skills.xml" \
      --output "./transformations_output/skills.ttl" \
      --format "TTL"

Here we merge our four Turtle files into one, which makes it easier to validate and upload the RDF in the next steps:

In [ ]:
!cat \
  ./transformations_output/distances.ttl \
  ./transformations_output/jobcategories-skills.ttl \
  ./transformations_output/jobcategories.ttl \
  ./transformations_output/skills.ttl \
  > ./transformations_output/data.ttl

## 4) Create SHACL shapes (RDF)

SHACL shapes describe what our RDF data must look like to be considered valid/conformant.

The RDF file (in Turtle) is in the `shapes` directory.



### Validate the RDF

The generated report gets saved in the `shapes_output` directory.



In [ ]:
!pyshacl \
  --shacl ./shapes/shapes.ttl \
  --format turtle \
  --output ./shapes_output/report.ttl \
  ./transformations_output/data.ttl

# Upload the knowledge graph

Thanks to the [SPARQL Protocol](https://www.w3.org/TR/sparql11-protocol/), we can upload RDF files to our triplestore using standard HTTP requests.

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./ontologies/o.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/ontologies/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./shapes/shapes.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/shacl-shapes/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./shapes_output/report.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/shacl-reports/"

In [ ]:
!curl -X "PUT" \
      -H "Content-Type:text/turtle" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      --data-binary "@./transformations_output/data.ttl" \
      "${SPARQL_ENDPOINT_STORE}/?graph=https://graphs.brox.de/dwt24/data/"

# Query the knowledge graph

## Named Graphs + count of instances (per class)

In [ ]:
!curl -X GET \
      -H "Accept:text/csv" \
      --user "${SPARQL_USER}:${SPARQL_PASSWORD}" \
      "${SPARQL_ENDPOINT_QUERY}" \
      --data-urlencode "query=SELECT DISTINCT ?g ?class (COUNT(DISTINCT ?s) AS ?count) WHERE { GRAPH ?g { ?s a ?class . } } GROUP BY ?g ?class ORDER BY ?g DESC(?count)"